This notebook samples from the GMM model fit in `../../train_gmm.ipynb` and `../../train_gmm_w_macro_holdouts.ipynb`.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import seaborn as sns
from scipy import stats
from tqdm.notebook import tqdm

from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture

sns.set_theme(style = 'dark')

#### Support functions

In [20]:
def wd_s_to_uv(ws, wd):
    """
    Translate wind speed and direction to (u,v).
    """
    return -ws * np.sin(wd * (np.pi / 180.)), -ws * np.cos(wd * (np.pi / 180.))

def wd_s_to_u(ws, wd):
    """
    Translate wind speed and direction to u.
    """
    return -ws * np.sin(wd * (np.pi / 180.))

def wd_s_to_v(ws, wd):
    """
    Translate wind speed and direction to v.
    """
    return -ws * np.cos(wd * (np.pi / 180.))

def uv_to_dir(u, v):
    """
    Wind components --> direction
    """
    sp = np.sqrt(u ** 2 + v ** 2)
    u_prime, v_prime = u / sp, v / sp
    return 360 - np.arccos(-v_prime) * (180 / np.pi)

def deg_to_dir(deg):
    """
    Translates from 360 degree to string direction
    """
    if deg <= 22.5:
        return 'NNE'
    elif deg <= 45.:
        return 'NE'
    elif deg <= 67.5:
        return 'ENE'
    elif deg <= 90.:
        return 'E'
    elif deg <= 112.5:
        return 'ESE'
    elif deg <= 135.:
        return 'SE'
    elif deg <= 157.5:
        return 'SSE'
    elif deg <= 180.:
        return 'S'
    elif deg <= 202.5:
        return 'SSW'
    elif deg <= 225.:
        return 'SW'
    elif deg <= 247.5:
        return 'WSW'
    elif deg <= 270.:
        return 'W'
    elif deg <= 292.5:
        return 'WNW'
    elif deg <= 315.:
        return 'NW'
    elif deg <= 337.5:
        return 'NNW'
    else:
        return 'Unknown'

def speed_num_to_str(speed):
    """
    Converted computed windspeed to str category
    """
    if speed <= 2.235:
        return '(-0.001, 2.235]'
    elif speed <= 5.364:
        return '(2.235, 5.364]'
    elif speed <= 8.047:
        return '(5.364, 8.047]'
    elif speed <= 15.646:
        return '(8.047, 15.646]'
    else:
        return 'Uncategorized'

# Load Models and Data

### Original data

In [6]:
DATA_DIR = '../../data/combined_macro_micro_wind_data.csv'

data = pd.read_csv(DATA_DIR)

# isolate U and V columns
u_cols = [f'u{i}' for i in np.arange(20, 255, 5)]
v_cols = [f'v{i}' for i in np.arange(20, 255, 5)]
macro_cols = ['macro_ws', 'macro_wd']

# big dataset
data_full = data[u_cols + v_cols + macro_cols]

# remove the empty columns and rows
# data.dropna(axis=1, how='all', inplace=True)
data_full.dropna(inplace=True)
print(data_full.shape)

data_u = data_full[u_cols].copy()
data_v = data_full[v_cols].copy()

# average over altitude for visualization
data_uv = np.zeros(shape=(data_u.shape[0], 2))
data_uv[:, 0] = data_u.values.mean(axis=1)
data_uv[:, 1] = data_v.values.mean(axis=1)

# define the indices for the u and v components
u_idxs = np.arange(47)
v_idxs = np.arange(47, 94)

# include macro data
macro_cols = [col for col in data_full.columns.values if 'macro' in col]
macro_cols_w = ['macro_ws', 'macro_wd']

data_macro = data_full[macro_cols].copy()

(6542, 96)


C:\Users\mcstanl1\AppData\Local\Temp\1\ipykernel_23036\1227225927.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_full.dropna(inplace=True)


In [7]:
def wd_s_to_uv(ws, wd):
    """
    Translate wind speed and direction to (u,v).
    """
    return -ws * np.sin(wd * (np.pi / 180.)), -ws * np.cos(wd * (np.pi / 180.))

def wd_s_to_u(ws, wd):
    """
    Translate wind speed and direction to u.
    """
    return -ws * np.sin(wd * (np.pi / 180.))

def wd_s_to_v(ws, wd):
    """
    Translate wind speed and direction to v.
    """
    return -ws * np.cos(wd * (np.pi / 180.))

# create dataframe for macro
data_wind_macro = data_full[u_cols + v_cols].copy()

# create columns for macro u and v
data_wind_macro['u_macro'] = data_full.apply(lambda x: wd_s_to_u(ws=x['macro_ws'], wd=x['macro_wd']), axis=1)
data_wind_macro['v_macro'] = data_full.apply(lambda x: wd_s_to_v(ws=x['macro_ws'], wd=x['macro_wd']), axis=1)
print(data_wind_macro.shape)

(6542, 96)


### Models

In [2]:
# GMM
with open('../../models/gmm.pkl','rb') as f:
    gmm = pickle.load(f)

# PCA
with open('../../models/pca.pkl','rb') as f:
    pca = pickle.load(f)

In [3]:
# PCA dimension-reduced data
with open('../../data/pca_dim_reduced_data.npy', 'rb') as f:
    data_pca = np.load(file=f)

### Hold-out models

In [13]:
# create lists of macro wind dir and speed
macro_wind_dirs = data['macro_wd_str'].value_counts().index.values[:4]
macro_wind_speeds = data['macro_ws_str'].value_counts().index.values
print(macro_wind_dirs)
print(macro_wind_speeds)

['WSW' 'SW' 'W' 'WNW']
['(-0.001, 2.235]' '(5.364, 8.047]' '(8.047, 15.646]' '(2.235, 5.364]']


In [14]:
# create 16 masks to filter out every combo of top 4 wind dir and wind speeds
macro_masks = np.zeros(shape=(data.shape[0], 16), dtype=bool)
dir_speed_pairs = {}

for i, dir in enumerate(macro_wind_dirs):
    for j, speed in enumerate(macro_wind_speeds):
        macro_masks[:, 4 * i + j] = ((data['macro_wd_str'] != dir) & (data['macro_ws_str'] != speed)).values
        dir_speed_pairs[4 * i + j] = (dir, speed)

In [15]:
# read in GMMs
gmm_hold_outs = []
for i in range(len(dir_speed_pairs)):

    with open(f'../../models/gmm_hold_out_models/{''.join(dir_speed_pairs[i])}.pkl', 'rb') as f:
        gmm_hold_outs.append(pickle.load(file=f))

In [17]:
# read in PCA objects
NUM_PCS = 7
NUM_GMM_COMPONENTS = 16
pca_vecs = np.zeros(shape=(len(dir_speed_pairs), NUM_PCS, data_wind_macro.shape[1]))
x_bars = np.zeros(shape=(len(dir_speed_pairs), data_wind_macro.shape[1]))

for i in range(len(dir_speed_pairs)):

    with open(f'../../models/gmm_hold_out_models/pca_obj_{''.join(dir_speed_pairs[i])}.npz', 'rb') as f:
        np_obj_i = np.load(file=f)
        pca_vecs[i] = np_obj_i['pca_vecs']
        x_bars[i] = np_obj_i['x_bars']

# Unconditional Samples

In [4]:
# set a number of PCs to use
NUM_PCS = 7

# save orthogongal transformation -- "h" stands for "high-dimensional"
orth_pca_vecs = pca.components_[:NUM_PCS]

In [8]:
# sample from GMM
data_sampled = gmm.sample(n_samples=data_pca.shape[0])[0]
print(data_sampled.shape)

# transform the sampled data back to the full space
data_sampled_reconstruct = data_sampled @ orth_pca_vecs + data_wind_macro.values.mean(axis=0)

(6542, 7)


In [10]:
# save data
with open('../../data/data_sampled_reconstruct.npy', 'wb') as f:
    np.save(file=f, arr=data_sampled_reconstruct)

# Conditional Samples

In [11]:
# compute microweather speeds for all altitudes
data_sampled_ws = np.zeros(shape=(data_sampled_reconstruct.shape[0], 47))
for i in range(47):
    data_sampled_ws[:, i] = np.sqrt(data_sampled_reconstruct[:, u_idxs[i]] ** 2 + data_sampled_reconstruct[:, v_idxs[i]] ** 2)

# make a numpy array of true windspeeds
ws_cols = [col for col in data.columns.values if 'ws' in col][:47]
data_ws = data[ws_cols].values.copy()
print(data_ws.shape)

# compute speed of sampled values
sampled_macro_ws = np.sqrt(data_sampled_reconstruct[:, -2] ** 2 + data_sampled_reconstruct[:, -1] ** 2)

# create index categories
category_names = ['(0, 2.23)', '[2.23, 5.36)', '[5.36, 8.05)', '[8.05, 15.65)']
category_idxs_sampled = {
    0: sampled_macro_ws < 2.23,
    1: (sampled_macro_ws >= 2.23) & (sampled_macro_ws < 5.36),
    2: (sampled_macro_ws >= 5.36) & (sampled_macro_ws < 8.05),
    3: (sampled_macro_ws > 8.05) & (sampled_macro_ws < 15.65)
}

# how many samples in each category?
for i in range(4):
    print(f'{category_names[i]}: {category_idxs_sampled[i].sum()}')

# create windspeed mask for true data
category_idxs_true = {
    0: data.macro_ws < 2.23,
    1: (data.macro_ws >= 2.23) & (data.macro_ws < 5.36),
    2: (data.macro_ws >= 5.36) & (data.macro_ws < 8.05),
    3: (data.macro_ws > 8.05) & (data.macro_ws < 15.65)
}

(7200, 47)
(0, 2.23): 2060
[2.23, 5.36): 1458
[5.36, 8.05): 1629
[8.05, 15.65): 1376


In [12]:
# save the wind speed data
with open('../../data/data_sampled_wind_speed.npy', 'wb') as f:
    np.save(file=f, arr=data_sampled_ws)

# Samples from hold-out models

In [18]:
# generate data
N = data_wind_macro.shape[0]
data_sampled = np.zeros(shape=(len(dir_speed_pairs), N, data_wind_macro.shape[1]))

for i in range(len(dir_speed_pairs)):

    # sample from gmm -- "ld" == "low-dimensional"
    sample_ld_i = gmm_hold_outs[i].sample(n_samples=N)[0]

    # project the sample to higher dimension
    data_sampled[i] = sample_ld_i @ pca_vecs[i] + x_bars[i]

In [21]:
# obtain macro wind dir and speed for each sample
sampled_macro_dir = np.zeros(shape=(16, N), dtype=np.object_)
sampled_macro_speed = np.zeros(shape=(16, N))

for i in range(16):
    for j in range(N):

        # direction
        u_j, v_j = data_sampled[i, j, -2:]
        sampled_macro_dir[i, j] = deg_to_dir(uv_to_dir(u=u_j, v=v_j))

        # speed
        sampled_macro_speed[i, j] = np.sqrt(u_j **2 + v_j ** 2)

In [22]:
# save data frame for each sample WITH macro dir and speed
for i in range(16):
    
    # generate the data frame
    df_i = pd.DataFrame(
        data_sampled[0],
        columns=u_cols + v_cols + ['macro_u', 'macro_v']
    )
    
    df_i['macro_wd_str'] = sampled_macro_dir[i]
    ws_i_ser = pd.Series(sampled_macro_speed[i])
    df_i['macro_ws_str'] = ws_i_ser.apply(speed_num_to_str)

    # save the data
    dir_i, speed_i = dir_speed_pairs[i]
    SAVE_PATH_I = f'../../data/gmm_hold_out/{dir_i}_{speed_i}.csv'
    df_i.to_csv(SAVE_PATH_I)